# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [ ]:
import timeit

In [2]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os
import pandas as pd

In [3]:
SEEDS = [10, 42, 4] # don't change!

In [4]:
### Training
def train(epoch, optimizer, criterion):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, seed):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if test_acc > best_acc:
        best_acc = test_acc
        print("==> Saving to checkpoint..")
        net.save(best_acc, epoch, seed)
    return test_acc, test_loss

In [5]:
resume = False
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 300 # number of iterations the model gets trained
learning_rate = 0.1 # factor for weight updates
learning_rate_switches = { # learning rate is reset after specific epochs
    '150': 0.01,
    '250': 0.001
}
momentum = 0.9
weight_decay = 5e-4

start = timeit.default_timer()

rows = []
for seed in SEEDS:
    for shuffle_setting in [{'train': False, 'test': False}, {'train': True, 'test': True}]:
        print("\n==> Beginning training with seed {} and shuffle setting {}".format(seed, shuffle_setting))
        print("-" * 30)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        print('==> Building model..')
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        print('==> Loading data..')
        dataloader = DataLoader(batch_sizes={'train': 64, 'test': 64}, shuffle=shuffle_setting)
        dataloader.download_cifar()
        trainloader, testloader = dataloader.get_loaders()

        if resume:
            print('==> Resuming from checkpoint..')
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/ckpt_{}.pth'.format(seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            if str(epoch+1) in learning_rate_switches.keys():
                print('==> Resetting learning rate')
                for param_group in optimizer.param_groups:
                    param_group['lr'] = learning_rate_switches[str(epoch+1)]
                    
            train_acc, train_loss = train(epoch, optimizer, criterion)
            test_acc, test_loss = test(epoch, test_acc, seed)
            print("[{}/{}]: Train Acc: {} | Test Acc: {} | Train Loss: {} | Test Loss: {}"\
                  .format(epoch+1, num_epochs, train_acc, test_acc, train_loss, test_loss))
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'shuffle': True if shuffle_setting['train'] == True else False,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'shuffle': True if shuffle_setting['train'] == True else False,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
            
stop = timeit.default_timer()
logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'shuffle', 'accuracy', 'loss'])        
logging_df.to_csv('training_logs.txt', sep='\t', index=False)
print(stop - start)


==> Beginning training with seed 10 and shuffle setting {'train': False, 'test': False}
------------------------------
==> Building model..
==> Loading data..
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Saving to checkpoint..
[1/300]: Train Acc: 30.204 | Test Acc: 42.21 | Train Loss: 0.029692338154315948 | Test Loss: 0.024461873602867126
==> Saving to checkpoint..
[2/300]: Train Acc: 47.41 | Test Acc: 54.88 | Train Loss: 0.022342565851211547 | Test Loss: 0.020094860738515855
==> Saving to checkpoint..
[3/300]: Train Acc: 61.142 | Test Acc: 62.8 | Train Loss: 0.017034988161325454 | Test Loss: 0.016805401432514192
==> Saving to checkpoint..
[4/300]: Train Acc: 70.042 | Test Acc: 68.6 | Train Loss: 0.013407085995078087 | Test Loss: 0.01476631572842598
==> Saving to checkpoint..
[5/300]: Train Acc: 74.12 | Test Acc: 74.18 | Train Loss: 0.011554079818725586 | Test Loss: 0.012024174857139587
==> Saving to checkpoint..
[6/300]: Train A

[63/300]: Train Acc: 84.456 | Test Acc: 79.68 | Train Loss: 0.007154313747435808 | Test Loss: 0.009137991181015968
==> Saving to checkpoint..
[64/300]: Train Acc: 84.56 | Test Acc: 82.22 | Train Loss: 0.007115069808065891 | Test Loss: 0.008532966715097427
[65/300]: Train Acc: 84.486 | Test Acc: 76.39 | Train Loss: 0.007089261915683746 | Test Loss: 0.011465628811717034
==> Saving to checkpoint..
[66/300]: Train Acc: 84.54 | Test Acc: 78.68 | Train Loss: 0.007164024459421635 | Test Loss: 0.010375807008147239
[67/300]: Train Acc: 84.398 | Test Acc: 77.29 | Train Loss: 0.007120339586436749 | Test Loss: 0.010807789206504822
==> Saving to checkpoint..
[68/300]: Train Acc: 84.514 | Test Acc: 80.71 | Train Loss: 0.00713280066549778 | Test Loss: 0.008897367417812347
[69/300]: Train Acc: 84.588 | Test Acc: 78.57 | Train Loss: 0.007102197563052177 | Test Loss: 0.009732581359148026
[70/300]: Train Acc: 84.48 | Test Acc: 76.84 | Train Loss: 0.007141555557847023 | Test Loss: 0.011013723051548005
==>

[127/300]: Train Acc: 84.692 | Test Acc: 77.76 | Train Loss: 0.007000757067501545 | Test Loss: 0.01033462873697281
==> Saving to checkpoint..
[128/300]: Train Acc: 84.956 | Test Acc: 80.11 | Train Loss: 0.006963993461132049 | Test Loss: 0.009009414859116077
==> Saving to checkpoint..
[129/300]: Train Acc: 84.682 | Test Acc: 80.72 | Train Loss: 0.007077049410045147 | Test Loss: 0.008867117609083652
[130/300]: Train Acc: 84.732 | Test Acc: 79.36 | Train Loss: 0.006974721609354019 | Test Loss: 0.009563758796453476
==> Saving to checkpoint..
[131/300]: Train Acc: 84.7 | Test Acc: 82.3 | Train Loss: 0.007016783650517464 | Test Loss: 0.008271797536313534
[132/300]: Train Acc: 84.854 | Test Acc: 77.39 | Train Loss: 0.007003361097872258 | Test Loss: 0.010534577694535256
==> Saving to checkpoint..
[133/300]: Train Acc: 84.91 | Test Acc: 79.51 | Train Loss: 0.006966587016880512 | Test Loss: 0.009603984582424164
==> Saving to checkpoint..
[134/300]: Train Acc: 84.626 | Test Acc: 80.85 | Train Los

==> Saving to checkpoint..
[190/300]: Train Acc: 96.94 | Test Acc: 90.93 | Train Loss: 0.001416290818527341 | Test Loss: 0.005013859637081623
[191/300]: Train Acc: 96.818 | Test Acc: 90.63 | Train Loss: 0.0014115072971582414 | Test Loss: 0.0051766605123877525
[192/300]: Train Acc: 96.988 | Test Acc: 90.59 | Train Loss: 0.001381943310946226 | Test Loss: 0.005429361417889595
==> Saving to checkpoint..
[193/300]: Train Acc: 96.884 | Test Acc: 90.83 | Train Loss: 0.0013956366735696792 | Test Loss: 0.00525816668793559
[194/300]: Train Acc: 97.254 | Test Acc: 90.38 | Train Loss: 0.0012799927805364133 | Test Loss: 0.0053899135690182445
==> Saving to checkpoint..
[195/300]: Train Acc: 96.788 | Test Acc: 91.36 | Train Loss: 0.0014493197614699601 | Test Loss: 0.004959633735194802
[196/300]: Train Acc: 97.212 | Test Acc: 91.23 | Train Loss: 0.001304693985953927 | Test Loss: 0.005114466837421059
[197/300]: Train Acc: 97.046 | Test Acc: 90.31 | Train Loss: 0.0013368735709786415 | Test Loss: 0.00560

==> Saving to checkpoint..
[253/300]: Train Acc: 99.782 | Test Acc: 94.05 | Train Loss: 0.00016435858950018882 | Test Loss: 0.003591726702079177
==> Saving to checkpoint..
[254/300]: Train Acc: 99.808 | Test Acc: 94.06 | Train Loss: 0.0001429077585786581 | Test Loss: 0.0036480036940425634
[255/300]: Train Acc: 99.826 | Test Acc: 93.99 | Train Loss: 0.0001257695323973894 | Test Loss: 0.0037085915580391884
==> Saving to checkpoint..
[256/300]: Train Acc: 99.85 | Test Acc: 94.03 | Train Loss: 0.00011956680737435818 | Test Loss: 0.003658270237967372
==> Saving to checkpoint..
[257/300]: Train Acc: 99.88 | Test Acc: 94.13 | Train Loss: 0.00010427257023751736 | Test Loss: 0.00367501832023263
==> Saving to checkpoint..
[258/300]: Train Acc: 99.916 | Test Acc: 94.18 | Train Loss: 8.816946782171727e-05 | Test Loss: 0.003650773920863867
[259/300]: Train Acc: 99.884 | Test Acc: 94.18 | Train Loss: 9.015406638383866e-05 | Test Loss: 0.00368080006018281
==> Saving to checkpoint..
[260/300]: Train A

...

[16/300]: Train Acc: 81.388 | Test Acc: 74.35 | Train Loss: 0.008576968996226788 | Test Loss: 0.013074357017874717
==> Saving to checkpoint..
[17/300]: Train Acc: 81.66 | Test Acc: 76.69 | Train Loss: 0.008429410828948021 | Test Loss: 0.012377232301235198
==> Saving to checkpoint..
[18/300]: Train Acc: 82.004 | Test Acc: 77.75 | Train Loss: 0.00829231546342373 | Test Loss: 0.010435507521033288
[19/300]: Train Acc: 82.0 | Test Acc: 77.35 | Train Loss: 0.008265236263871193 | Test Loss: 0.010253204944729805
[20/300]: Train Acc: 82.122 | Test Acc: 75.84 | Train Loss: 0.008292598156929017 | Test Loss: 0.012158423227071762
==> Saving to checkpoint..
[21/300]: Train Acc: 82.068 | Test Acc: 78.39 | Train Loss: 0.008231553330421448 | Test Loss: 0.010142400142550469
[22/300]: Train Acc: 82.498 | Test Acc: 72.06 | Train Loss: 0.00803017351925373 | Test Loss: 0.013579055216908454
==> Saving to checkpoint..
[23/300]: Train Acc: 82.624 | Test Acc: 77.1 | Train Loss: 0.007918070051968098 | Test Loss:

==> Saving to checkpoint..
[81/300]: Train Acc: 84.186 | Test Acc: 81.0 | Train Loss: 0.007256455851197243 | Test Loss: 0.008805266234278679
[82/300]: Train Acc: 84.442 | Test Acc: 76.38 | Train Loss: 0.007219163457751274 | Test Loss: 0.013190251019597053
==> Saving to checkpoint..
[83/300]: Train Acc: 84.36 | Test Acc: 79.55 | Train Loss: 0.007231299472153186 | Test Loss: 0.010293285429477692
[84/300]: Train Acc: 84.472 | Test Acc: 76.03 | Train Loss: 0.00723273849695921 | Test Loss: 0.011642130902409554
==> Saving to checkpoint..
[85/300]: Train Acc: 84.378 | Test Acc: 82.62 | Train Loss: 0.007212449252605438 | Test Loss: 0.008297304019331932
[86/300]: Train Acc: 84.376 | Test Acc: 76.09 | Train Loss: 0.00723739786028862 | Test Loss: 0.012620506072044372
[87/300]: Train Acc: 84.268 | Test Acc: 74.16 | Train Loss: 0.007220349144637585 | Test Loss: 0.013529431375861168
==> Saving to checkpoint..
[88/300]: Train Acc: 84.638 | Test Acc: 75.16 | Train Loss: 0.007133446907401085 | Test Los

[144/300]: Train Acc: 84.722 | Test Acc: 82.04 | Train Loss: 0.007113115373849869 | Test Loss: 0.008272521317005158
==> Saving to checkpoint..
[145/300]: Train Acc: 84.716 | Test Acc: 85.62 | Train Loss: 0.00708162849932909 | Test Loss: 0.006739298126101494
[146/300]: Train Acc: 84.532 | Test Acc: 75.8 | Train Loss: 0.007073948925733566 | Test Loss: 0.012241432029008865
==> Saving to checkpoint..
[147/300]: Train Acc: 84.532 | Test Acc: 81.4 | Train Loss: 0.007098220073282719 | Test Loss: 0.009011333048343659
[148/300]: Train Acc: 84.582 | Test Acc: 73.3 | Train Loss: 0.007042335081100464 | Test Loss: 0.01370875951051712
==> Saving to checkpoint..
[149/300]: Train Acc: 84.324 | Test Acc: 77.03 | Train Loss: 0.00719552803426981 | Test Loss: 0.011603185042738915
==> Resetting learning rate
==> Saving to checkpoint..
[150/300]: Train Acc: 91.24 | Test Acc: 91.58 | Train Loss: 0.004045798174440861 | Test Loss: 0.0038804636724293234
==> Saving to checkpoint..
[151/300]: Train Acc: 93.412 | 

==> Saving to checkpoint..
[207/300]: Train Acc: 96.646 | Test Acc: 91.67 | Train Loss: 0.0015233996574580669 | Test Loss: 0.004360990591347217
[208/300]: Train Acc: 96.79 | Test Acc: 91.49 | Train Loss: 0.0014865304777771234 | Test Loss: 0.0045833853837102655
[209/300]: Train Acc: 96.906 | Test Acc: 91.17 | Train Loss: 0.0014462255939096213 | Test Loss: 0.0045220427557826045
[210/300]: Train Acc: 96.85 | Test Acc: 90.92 | Train Loss: 0.001450917810946703 | Test Loss: 0.0046901576064527035
==> Saving to checkpoint..
[211/300]: Train Acc: 96.612 | Test Acc: 92.2 | Train Loss: 0.0015149651987105609 | Test Loss: 0.004130402305349707
[212/300]: Train Acc: 96.896 | Test Acc: 92.18 | Train Loss: 0.00143641368791461 | Test Loss: 0.0042501591928303245
[213/300]: Train Acc: 97.042 | Test Acc: 91.35 | Train Loss: 0.0013898256272822619 | Test Loss: 0.0047694294519722465
[214/300]: Train Acc: 96.796 | Test Acc: 91.11 | Train Loss: 0.0014691925644874572 | Test Loss: 0.00460609991364181
[215/300]: T

==> Saving to checkpoint..
[270/300]: Train Acc: 99.92 | Test Acc: 94.86 | Train Loss: 7.315126664936542e-05 | Test Loss: 0.0031517072945833205
==> Saving to checkpoint..
[271/300]: Train Acc: 99.942 | Test Acc: 94.98 | Train Loss: 6.42569849640131e-05 | Test Loss: 0.0030883808258920908
==> Saving to checkpoint..
[272/300]: Train Acc: 99.942 | Test Acc: 94.99 | Train Loss: 6.281428821384907e-05 | Test Loss: 0.003091538267955184
[273/300]: Train Acc: 99.944 | Test Acc: 94.95 | Train Loss: 6.351725332438946e-05 | Test Loss: 0.003164624970406294
==> Saving to checkpoint..
[274/300]: Train Acc: 99.93 | Test Acc: 95.0 | Train Loss: 6.183858960866927e-05 | Test Loss: 0.003092853631824255
==> Saving to checkpoint..
[275/300]: Train Acc: 99.946 | Test Acc: 95.06 | Train Loss: 5.7828730940818786e-05 | Test Loss: 0.003077480700239539
[276/300]: Train Acc: 99.95 | Test Acc: 94.95 | Train Loss: 5.499748170375824e-05 | Test Loss: 0.003193405508622527
[277/300]: Train Acc: 99.938 | Test Acc: 94.83 |